Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [2]:
from itertools import combinations
from collections import deque
import numpy as np
import random
from geopy.distance import geodesic
import networkx as nx

import pandas as pd
import logging
import pickle
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

from typing import Tuple

logging.basicConfig(level=logging.DEBUG)

# Initialization

In [3]:
def init_all_cities() -> Tuple[dict, dict]:
    ALL_CITIES = {}
    ALL_DIST_MATRIX = {}
    countries = []

    for country in os.listdir('cities'):
        if country.endswith('.csv'):
            country = country[:-4]
            countries.append(country)

    for country in countries:
        # Read the CSV file
        ALL_CITIES[country] = pd.read_csv(f'cities/{country}.csv', header=None, names = ['name', 'lat', 'lon'])

        # Calculate distances between all cities
        cur_dist_matrix = np.zeros((len(ALL_CITIES[country]), len(ALL_CITIES[country])))
        for c1, c2 in combinations(ALL_CITIES[country].itertuples(), 2):
            cur_dist_matrix[c1.Index, c2.Index] = cur_dist_matrix[c2.Index, c1.Index] = geodesic((c1.lat, c1.lon), (c2.lat, c2.lon)).km
        ALL_DIST_MATRIX[country] = cur_dist_matrix

    return ALL_CITIES, ALL_DIST_MATRIX

ALL_CITIES, ALL_DIST_MATRIX = init_all_cities()

In [9]:
def initialize(country: str):
    CITIES = ALL_CITIES[country]
    DIST_MATRIX = ALL_DIST_MATRIX[country]

    return CITIES, DIST_MATRIX

COUNTRY = 'vanuatu'
CITIES, DIST_MATRIX = initialize(COUNTRY)

logging.info(f'Loaded {len(CITIES)} cities in {COUNTRY}')

INFO:root:Loaded 8 cities in vanuatu


# Helper Functions

### General

In [5]:
def tsp_cost(tsp: list[int]) -> float:
    # Check that the TSP is a cycle and that all cities have been covered
    assert tsp[0] == tsp[-1], f"first city is #{tsp[0]}, last city is #{tsp[-1]}"
    # logging.debug(f"set(tsp): {set(tsp)}\nset(range(len(tsp))): {set(range(len(tsp)))}")
    assert set(tsp) == set(range(len(tsp) - 1)), f"tsp covers {len(tsp)}, should cover {len(tsp) - 1}"

    # Finally, compute the total cost
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]

    return tot_cost

def print_tsp(tsp: list):
    first = True
    for city in tsp:
        if first:
            prev_city = city
            first = False
            continue

        logging.info(f"step: {CITIES.at[prev_city,'name']} -> {CITIES.at[city,'name']} ({DIST_MATRIX[prev_city, city]:.2f}km)")
        prev_city = city

    logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")



#### Mutations Helper

In [11]:
def swap_mutation(individual):
    idx1, idx2 = random.sample(range(len(individual)), 2)
    individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

def scramble_mutation(individual, strength=0.4):
    num_elements = int(strength * len(individual))
    if num_elements <= 2:
        num_elements = 2

    indices = random.sample(range(len(individual)), num_elements)
    subset = [individual[i] for i in indices]
    random.shuffle(subset)
    for i, idx in enumerate(indices):
        individual[idx] = subset[i]
    return individual

def insert_mutation(individual):
    idx1, idx2 = random.sample(range(len(individual)), 2)
    gene = individual.pop(idx1)
    individual.insert(idx2, gene)
    return individual

def inversion_mutation(individual, strength=0.4):
    num_elements = int(strength * len(individual))
    if num_elements <= 2:
        num_elements = 2

    start_idx = random.randint(0, len(individual) - 1)

    # Create a circular buffer and rotate
    circular_buffer = deque(individual)
    circular_buffer.rotate(-start_idx)

    # Invert the selected number of elements
    subset = list(circular_buffer)[:num_elements]
    subset.reverse()
    for i in range(num_elements):
        circular_buffer[i] = subset[i]

    # Rotate back to the original position and revert to list
    circular_buffer.rotate(start_idx)
    individual[:] = list(circular_buffer)

    return individual

#### Crossover Helper

In [7]:
def cycle_crossover(parent1, parent2):
    size = len(parent1)
    child = [None] * size
    cycle_start = 0

    # Create one cycle. Save the genes of the cycle from parent 1
    complete = False
    while not complete:
        idx = cycle_start
        while True:
            # Copy the gene from parent1 to the child, then go on with the cycle exploration
            child[idx] = parent1[idx]
            idx = parent1.index(parent2[idx])
            # logging.debug(f"{child[idx]}) -> ")
            if idx == cycle_start:
                complete = True
                break
    
    # Fill in the remaining positions with genes from parent2
    for i in range(size):
        if child[i] is None:
            child[i] = parent2[i]
    
    return child

def inver_over_crossover(parent1, parent2):
    """ This implementation of the inver over refers to the basic version seen in https://www.sciencedirect.com/science/article/pii/S0898122111005530,
        but avoids the mutation chance and just mixes the parents, as I implement mutation separately."""
    idx1 = random.randint(0, len(parent1) - 1)
    city_i = parent1[idx1]
    idx2 = parent2.index(city_i)
    city_j = parent2[idx2 + 1] if idx2 < len(parent2) - 1 else parent2[0]
    logging.debug(f"city_i: {city_i} at position {idx1}, city_j: {city_j} at position {idx2}")
    
    # Perform inversion if city_i and city_j are not adjacent
    idx2 = parent1.index(city_j)
    if abs(idx1 - idx2) > 1:
        # Invert the section between the two cities
        if idx1 < idx2:
            parent1[idx1:idx2+1] = reversed(parent1[idx1:idx2+1])
        else:
            parent1[idx2:idx1+1] = reversed(parent1[idx2:idx1+1])
    
    child = parent1
    return child

# Evolutionary and Genetic Algorithms  
In the following all implementations will use Modern Flow for Genetic Algorithms, except for the last one which uses the Hypermodern Flow.  

## Starting From A Greedy Solution (Single Individual)
The following solutions will start from a population derived from somewhat wide mutations of Greedy Solution #2.

In the following I will use scramble and inversion as mutations because I feel like others may prove too small of a variation, since we are starting to mutate from just one solution. Inversion may not be enough actually but I'll try.  

Inversion doesn't seem to make sense with inner over, as the chances of mantaining adjacency is high with Inversion.  

Understanding how the various algorithms work takes enough time to not allow me to explore them all, so I'll stick with Inver Over (because it's the one used in the best tsp solution ever recorded) and Cycle (because I dind't manage to study Partially Mapped Crossover too).

#### Create the population

In [21]:
starting_mutations_composition = {swap_mutation: 0.1,
                                  scramble_mutation: 0.3,
                                  inversion_mutation: 0.3,
                                  insert_mutation: 0.3}

with open(f'./pickles/{COUNTRY}_tsp_greedy2.pkl', 'rb') as file:
    tsp = pickle.load(file)

population = [tsp]
population_size = 1000

for mutation, strength in starting_mutations_composition.items():
    for _ in range(int(population_size * strength)):
        individual = tsp.copy()
        individual = mutation(individual)
        population.append(individual)
population = population[:population_size]

logging.info(f"Generated initial population of {len(population)} individuals")
# logging.info(f"Initial population:")
# for i, individual in enumerate(population):
#     logging.debug(f"{i:5d}) {individual}")


INFO:root:Generated initial population of 1000 individuals


### Scramble + Inver Over  

### Scramble + Cycle

### Inversion + Cycle

## Starting From Random Population

### Swap + Cycle

### Swap + Inver Over

### Scramble + Cycle

### Scramble + Inver Over

### Insert + Cycle

### Insert + Inver Over

### Inversion + Cycle

### Inversion + Inver Over

## Full Inver Over (Hypermodern?)

#### Tests

In [ ]:
# Checking if the contents of the pickles are the same as the ones in the previous cells
with open(f'./pickles/{COUNTRY}_tsp_greedy1.pkl', 'rb') as file:
    tsp1 = pickle.load(file)
# assert tsp1 == tsp_greedy_1, f"pkl for tsp1 isn't the same as when written"

with open(f'./pickles/{COUNTRY}_tsp_greedy2.pkl', 'rb') as file:
    tsp2 = pickle.load(file)
# assert tsp == tsp_greedy_2, f"pkl for tsp2 isn't the same as when written"

# Calculate costs
logging.debug(f"cost1: {tsp_cost(tsp1)}, cost2: {tsp_cost(tsp2)}")

tsp1 = tsp1[:-1]
tsp2 = tsp2[:-1]

# rotate the tsp2 to start at the same city as tsp1
idx = tsp2.index(tsp1[0])
tsp2 = tsp2[idx:] + tsp2[:idx]

# logging.info(f"tsp2 is                      {tsp2}")
# logging.info(f"inversion_mutation(tsp2) is  {inversion_mutation(tsp2, 0.5)}")
# logging.info(f"insert_mutation(tsp2) is     {insert_mutation(tsp2)}")
# logging.info(f"scramble_mutation(tsp2) is   {scramble_mutation(tsp2, 0.5)}")
# logging.info(f"swap_mutation(tsp2) is       {swap_mutation(tsp2)}")

# logging.info(f"tsp1 is                                      {tsp1}")
# logging.info(f"tsp2 is                                      {tsp2}")
# logging.info(f"inver_over_crossover(tsp1, tsp2) is          {inver_over_crossover(tsp1, tsp2)}")
# logging.info(f"cycle_crossover(tsp1, tsp2) is               {cycle_crossover(tsp1, tsp2)}")

DEBUG:root:cost1: 1475.528091104531, cost2: 1475.5280911045313
